In [1]:
import pandas as pd
import datetime

#api
import praw

#pre-processing
import nltk 
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import MWETokenizer 
from nltk import word_tokenize
import re
# import string
#from textblob import TextBlob
#from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
# nltk.download()

from NLP_functions.py import display_topics, prob_df, topic_matrix

#analysis
from sklearn.decomposition import TruncatedSVD, NMF

ModuleNotFoundError: No module named 'NLP_functions'

In [ ]:
reddit = praw.Reddit(client_id='oCxykEUclueAuA', client_secret='M8wVQw3nFxMsXHLGxurnTtkN-Mdvtg', user_agent='Crypto')

#### Get top posts of all time

In [ ]:
posts = []
ml_subreddit = reddit.subreddit('Cryptocurrency')

for post in list(ml_subreddit.top('all', limit=1000)):
    date = datetime.datetime.utcfromtimestamp(post.created_utc)
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, date])
    
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'date'])
print(posts)

#### Get top coments from each top post

In [ ]:
top_comments = []
for i,d in zip(posts.id, posts.date):
    try:
        submission = reddit.submission(id=i)
        submission.comments.replace_more(limit=0)
        comments=list([(comment.body) for comment in submission.comments.list()])
        comment_string = ''
        for x in comments[:20]:
            comment_string = comment_string + ' ' + x
        top_comments.append([comment_string,d])
    except:
        comments=None
top_comments = pd.DataFrame(top_comments,columns=['comment', 'date'])
print(top_comments)

## Cleaning

combine comments by year

In [ ]:
['year'] = top_comments['date'].dt.year
years = ['2017', '2018', '2019', '2020', '2021']
top_comments['year'].value_counts()

In [ ]:
comment_year = pd.DataFrame(index=years, columns=['Comments'])
for y in years:
    comment_string = ''
    for index, row in top_comments.iterrows():
        if str(row[2]) == y:
            comment_string = comment_string + ' ' + row[0]
            
    comment_string = re.sub(r'\([^)]*\)', '', comment_string) #remove links
    comment_string = re.sub('\w*\d\w*', '', comment_string) #remove numbers
    comment_string = re.sub(r'[^\w\s]', '', (comment_string))#remove punctuation
    comment_string = comment_string.lower() #lowercase
    comment_string = comment_string.replace('btc', 'bitcoin') #replace bitcoin acronym 
    
    #create stop words list
    stop = stopwords.words('english')
    stop += [ 'moderator', 'subreddit', 'reddit' , 'question', 'concern', 'please', 'contact', 'im', 
            'people', 'gon', 'seems', 'something', 'year', 'like', 'mod']
    stop = set(stop)

    mwe_tokenizer = MWETokenizer([ ('bull','run'), ('market','cap'), ('pump','dump'),
                             ('stock','market'), ('flaired', 'inaccurately'), ('get', 'rich', 'quick')])
    
    comment_token = mwe_tokenizer.tokenize(word_tokenize(comment_string))  # tokenize words

    #lemmatize words
    lemmatizer = nltk.stem.WordNetLemmatizer()
    comment_token_lemm = [(lemmatizer.lemmatize(w)) for w in comment_token]

    comment_token_lemm = [w for w in comment_token_lemm if w not in stop] #remove stop words
    
    comment_token_lemm = " ".join(comment_token_lemm)
    
    comment_year.loc[y, 'Comments'] = comment_token_lemm

In [ ]:
print(comment_year)

In [ ]:
comment_year = re.sub(r'\([^)]*\)', '', all_comments) #remove links
comment_year = re.sub('\w*\d\w*', '', all_comments) #remove numbers
comment_year = re.sub(r'[^\w\s]', '', (all_comments))#remove punctuation
comment_year = comment_year.lower() #lowercase
comment_year = comment_year.replace('btc', 'bitcoin') #replace bitcoin acronym 

In [ ]:
all_comments = comment_year.loc['2017', 'Comments'] #concatenate comments column into one string

all_comments = re.sub(r'\([^)]*\)', '', all_comments) #remove links
all_comments = re.sub('\w*\d\w*', '', all_comments) #remove numbers
all_comments = re.sub(r'[^\w\s]', '', (all_comments))#remove punctuation
all_comments = all_comments.lower() #lowercase
all_comments = all_comments.replace('btc', 'bitcoin') #replace bitcoin acronym 

#create stop words list
stop = stopwords.words('english')
stop += [ 'moderator', 'subreddit', 'reddit' , 'question', 'concern', 'please', 'contact', 'im', 
        'people', 'gon', 'seems', 'something', 'year', 'like', 'mod']
stop = set(stop)

mwe_tokenizer = MWETokenizer([ ('bull','run'), ('market','cap'), ('pump','dump'),
                             ('stock','market'), ('flaired', 'inaccurately'), ('get', 'rich', 'quick')])

words = mwe_tokenizer.tokenize(word_tokenize(all_comments))  # tokenize words

#lemmatize words
lemmatizer = nltk.stem.WordNetLemmatizer()
words = [(lemmatizer.lemmatize(w)) for w in words]

words = [w for w in words if w not in stop] #remove stop words

#create n grams
n = 2
bigrams = ngrams(words, n)

#count phrases
counter = Counter()
counter += Counter(bigrams)

for word, count in counter.most_common(30):
    print('%20s %i' % (" ".join(word), count))

In [ ]:
new = s.apply(fun) 

# Topic Modeling

### Count Vectorization

In [ ]:
comments = list(comment_year['Comments'])

#comments_cleaned = [re.sub('\w*\d\w*', '', i) for i in comments]

# set(stopwords.words('english'))
cv = CountVectorizer(stop_words=stop)
X= cv.fit_transform(comments)
pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

#### Latent Semantic Analysis (LSA) with Count Vectorization

In [ ]:
lsa = TruncatedSVD(3)
doc_topic = lsa.fit_transform(X)
lsa.explained_variance_ratio_

In [ ]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2", "component_3"],
             columns = cv.get_feature_names())
topic_word

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
display_topics(lsa, cv.get_feature_names(), 5)

In [ ]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = ['2017', '2018', '2019', '2020', '2021']s,
             columns = ["component_1","component_2", "component_3" ])
Vt

#### Non-Negative Matrix Factorization (NMF)

In [ ]:
nmf_model = NMF(3)
X_topic = nmf_model.fit_transform(X)

In [ ]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2", "component_3"],
             columns = cv.get_feature_names())
topic_word

In [ ]:
display_topics(nmf_model, cv.get_feature_names(), 10)

In [ ]:
H = pd.DataFrame(doc_topic.round(5),
             index = comments,
             columns = ["component_1","component_2", "component_3" ])
H